# Reconstructing the full ϒ(4S) event
In this part of the lesson, we will reconstruct the full ϒ(4S) event using the FEI. The B0 decay process we will be looking at is the semileptonic decay to a charged pion, a muon and a muon neutrino.

In [1]:
import basf2 as b2
import modularAnalysis as ma
from variables import variables as vm

Welcome to JupyROOT 6.20/04


Import a dataset with a pre-applied FEI (known as "skim"s) which already contains constructed `B0:generic` particles.

In [2]:
main = b2.Path()

ma.inputMdst(
    "default",
    b2.find_file("starterkit/2021/fei_skimmed_xulnu.udst.root", "examples"),
    path=main,
)

Now, fill the particle lists with muons and charged pions

In [3]:
good_track = (
    " and dr < 0.5 and abs(dz) < 2 and nCDCHits > 20 and thetaInCDCAcceptance"
)

ma.fillParticleList("mu-", "muonID > 0.9" + good_track, path=main)
ma.fillParticleList("pi-", "pionID > 0.5" + good_track, path=main)

Now, we will create the signal B meson. As this semileptonic decay contains a neutrino (which can't be seen by the Belle 2 detector), we need to tell the MC matching algorithm to accept missing neutrinos to avoid discrepancy between the simulated and reconstructed decays (causing `isSignal` to be zero).

In [4]:
ma.reconstructDecay("B0:signal -> pi- mu+ ?nu", cut="", path=main)

Now we can use the reconstructed B0 signal (in fact, we will use it's anti-particle) and the B0 from the FEI `B0:generic` to reconstruct the Upsilon(4S):

In [5]:
ma.reconstructDecay(
    "Upsilon(4S):opposite_cp -> B0:generic anti-B0:signal", cut="", path=main
)
ma.reconstructDecay(
    decayString="Upsilon(4S):same_cp -> B0:generic B0:signal",
    cut="",
    path=main,
)
# Combine the two Upsilon(4S) lists to one. Note: Duplicates are removed.
ma.copyLists(
    outputListName="Upsilon(4S)",
    inputListNames=["Upsilon(4S):opposite_cp", "Upsilon(4S):same_cp"],
    path=main,
)

Re-create the Rest of Event (ROE) around the newly reconstructed Upsilon(4S)

In [6]:
ma.buildRestOfEvent("Upsilon(4S)", path=main)
track_based_cuts = "thetaInCDCAcceptance and pt > 0.075 and dr < 2 and abs(dz) < 4"
ecl_based_cuts = "thetaInCDCAcceptance and E > 0.05"
roe_mask = ("my_mask", track_based_cuts, ecl_based_cuts)
ma.appendROEMasks("Upsilon(4S)", [roe_mask], path=main)

Finally, we add MC matching to the Upsilon(4S) and we will add useful variables to see how well the reconstruction process went

In [7]:
ma.matchMCTruth(list_name="Upsilon(4S)", path=main)

vm.addAlias("Btag_SigProb", "daughter(0, extraInfo(SignalProbability))")
vm.addAlias("Btag_decayModeID", "daughter(0, extraInfo(decayModeID))")
vm.addAlias("Btag_Mbc", "daughter(0, Mbc)")

vm.addAlias("Bsig_isSignal",
            "daughter(1, isSignal)")
vm.addAlias("nCharged", "nROE_Charged(my_mask)")

ma.variablesToNtuple(
    "Upsilon(4S)",
    variables=[
        "Btag_SigProb",
        "Btag_decayModeID",
        "Btag_Mbc",
        "Bsig_isSignal",
        "isSignal",
        "m2RecoilSignalSide",
        "nCharged"
    ],
    filename='Upsilon4S.root',
    path=main,
)

In [8]:
# Process events
b2.process(main)